In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
appName("itv009033_week08"). \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv009033/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

# About Dataset:

Uncover the factors that lead to employee attrition and explore important questions such as ‘show me a breakdown of distance from home by job role and attrition’ or ‘compare average monthly income by education and attrition’. This is a fictional data set created by IBM data scientists.

## Education
1. 'Below College'
2. 'College'
3. 'Bachelor'
4. 'Master'
5. 'Doctor'

## EnvironmentSatisfaction
1. 'Low'
2. 'Medium'
3. 'High'
4. 'Very High'

## JobInvolvement
1. 'Low'
2. 'Medium'
3. 'High'
4. 'Very High'

## JobSatisfaction
1. 'Low'
2. 'Medium'
3. 'High'
4. 'Very High'

## PerformanceRating
1. 'Low'
2. 'Good'
3. 'Excellent'
4. 'Outstanding'

## RelationshipSatisfaction
1. 'Low'
2. 'Medium'
3. 'High'
4. 'Very High'

## WorkLifeBalance
1. 'Bad'
2. 'Good'
3. 'Better'
4. 'Best'

https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/data

# Question 3.
<h3>Select a relevant dataset of your choice and write some queries to
demonstrate the following :
- Running total, Grouping aggregates and various Window functions like
rank, dense_rank, row_num, lead, lag.
- Also try creating a pivot view.</h3>

In [2]:
file_path = "week08/WA_Fn-UseC_-HR-Employee-Attrition.csv"

In [3]:
hr_schema = "Age int, Attrition string, BusinessTravel string,DailyRate long,Department string,DistanceFromHome int,Education int,EducationField string,EmployeeCount int,EmployeeNumber int,EnvironmentSatisfaction string,Gender string,HourlyRate int,JobInvolvement int,JobLevel int,JobRole string,JobSatisfaction int,MaritalStatus string,MonthlyIncome long,MonthlyRate long,NumCompaniesWorked int,Over18 string,OverTime string,PercentSalaryHike int,PerformanceRating int,RelationshipSatisfaction int,StandardHours int,StockOptionLevel int,TotalWorkingYears int,TrainingTimesLastYear int,WorkLifeBalance int,YearsAtCompany int,YearsInCurrentRole int,YearsSinceLastPromotion int,YearsWithCurrManager int"

In [4]:
hr_df = spark.read.format("csv").schema(hr_schema).load(file_path)

In [5]:
hr_df.show(2,truncate = False)

+---+---------+-----------------+---------+----------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|BusinessTravel   |DailyRate|Department            |DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|JobRole           |JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

In [6]:
hr_df = hr_df.drop('Over18', 'EmployeeNumber','EmployeeCount','StandardHours')

In [7]:
hr_df.show()

+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+--------+-----------------+-----------------+------------------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSinceLastPromotion|YearsWithCurrManager|
+---+---------+---

## Attritions

In [8]:
att_df =  hr_df.groupBy("Attrition").count().withColumnRenamed("count", "count_of_attritions").sort("count_of_attritions")

In [9]:
att_df.show()

+---------+-------------------+
|Attrition|count_of_attritions|
+---------+-------------------+
|      Yes|                237|
|       No|               1233|
+---------+-------------------+



In [10]:
att = hr_df.filter((hr_df.Attrition == "Yes"))
att.show()

+---+---------+-----------------+---------+--------------------+----------------+---------+----------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+--------+-----------------+-----------------+------------------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|  EducationField|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSinceLastPromotion|YearsWithCurrManager|
+---+---------

## Attritions Per department

In [11]:
att_dept = hr_df.filter((hr_df.Attrition == "Yes"))
att_dept.show()

+---+---------+-----------------+---------+--------------------+----------------+---------+----------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+--------+-----------------+-----------------+------------------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|  EducationField|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSinceLastPromotion|YearsWithCurrManager|
+---+---------

In [12]:
att_dept = att_dept.groupBy("Department", "Attrition").count().withColumnRenamed("count", "count_of_attritions_per_dept").sort("count_of_attritions_per_dept", ascending =  False)

In [13]:
att_dept.show(truncate= False)

+----------------------+---------+----------------------------+
|Department            |Attrition|count_of_attritions_per_dept|
+----------------------+---------+----------------------------+
|Research & Development|Yes      |133                         |
|Sales                 |Yes      |92                          |
|Human Resources       |Yes      |12                          |
+----------------------+---------+----------------------------+



In [14]:
att_gender = att.groupBy("gender", "Attrition").count().withColumnRenamed("count", "count_of_attritions_per_gender").sort("count_of_attritions_per_gender", ascending =  False)

In [15]:
att_gender.show()

+------+---------+------------------------------+
|gender|Attrition|count_of_attritions_per_gender|
+------+---------+------------------------------+
|  Male|      Yes|                           150|
|Female|      Yes|                            87|
+------+---------+------------------------------+



In [16]:
att_role = att.groupBy("OverTime", "Attrition", "Attrition").count().withColumnRenamed("count", "count_of_attritions_per_gender").sort("count_of_attritions_per_gender", ascending =  False)
att_role.show(truncate = False)

+--------+---------+---------+------------------------------+
|OverTime|Attrition|Attrition|count_of_attritions_per_gender|
+--------+---------+---------+------------------------------+
|Yes     |Yes      |Yes      |127                           |
|No      |Yes      |Yes      |110                           |
+--------+---------+---------+------------------------------+



In [17]:
att_time = att.groupBy("OverTime", "Attrition").count().withColumnRenamed("count", "count_of_attritions_per_gender").sort("count_of_attritions_per_gender", ascending =  False)
att_time.show()

+--------+---------+------------------------------+
|OverTime|Attrition|count_of_attritions_per_gender|
+--------+---------+------------------------------+
|     Yes|      Yes|                           127|
|      No|      Yes|                           110|
+--------+---------+------------------------------+



In [18]:
salary = hr_df.select("MonthlyRate", "Attrition").sort("MonthlyRate")
salary.show()

+-----------+---------+
|MonthlyRate|Attrition|
+-----------+---------+
|       2094|       No|
|       2097|       No|
|       2104|       No|
|       2112|       No|
|       2122|       No|
|       2125|       No|
|       2125|       No|
|       2137|       No|
|       2227|       No|
|       2243|       No|
|       2253|       No|
|       2261|       No|
|       2288|       No|
|       2302|       No|
|       2323|       No|
|       2326|      Yes|
|       2338|       No|
|       2354|       No|
|       2373|       No|
|       2396|      Yes|
+-----------+---------+
only showing top 20 rows



In [19]:
final_df = hr_df.select("OverTime","OverTime","gender", "Department","MonthlyIncome","MonthlyRate","Attrition").cache()

In [20]:
final_df.show(truncate = False)

+--------+--------+------+----------------------+-------------+-----------+---------+
|OverTime|OverTime|gender|Department            |MonthlyIncome|MonthlyRate|Attrition|
+--------+--------+------+----------------------+-------------+-----------+---------+
|Yes     |Yes     |Female|Sales                 |5993         |19479      |Yes      |
|No      |No      |Male  |Research & Development|5130         |24907      |No       |
|Yes     |Yes     |Male  |Research & Development|2090         |2396       |Yes      |
|Yes     |Yes     |Female|Research & Development|2909         |23159      |No       |
|No      |No      |Male  |Research & Development|3468         |16632      |No       |
|No      |No      |Male  |Research & Development|3068         |11864      |No       |
|Yes     |Yes     |Female|Research & Development|2670         |9964       |No       |
|No      |No      |Male  |Research & Development|2693         |13335      |No       |
|No      |No      |Male  |Research & Development|9526 

In [21]:
final_df = final_df.withColumn("profit", final_df.MonthlyRate - final_df.MonthlyIncome)

In [22]:
final_df.show(truncate =  False)

+--------+--------+------+----------------------+-------------+-----------+---------+------+
|OverTime|OverTime|gender|Department            |MonthlyIncome|MonthlyRate|Attrition|profit|
+--------+--------+------+----------------------+-------------+-----------+---------+------+
|Yes     |Yes     |Female|Sales                 |5993         |19479      |Yes      |13486 |
|No      |No      |Male  |Research & Development|5130         |24907      |No       |19777 |
|Yes     |Yes     |Male  |Research & Development|2090         |2396       |Yes      |306   |
|Yes     |Yes     |Female|Research & Development|2909         |23159      |No       |20250 |
|No      |No      |Male  |Research & Development|3468         |16632      |No       |13164 |
|No      |No      |Male  |Research & Development|3068         |11864      |No       |8796  |
|Yes     |Yes     |Female|Research & Development|2670         |9964       |No       |7294  |
|No      |No      |Male  |Research & Development|2693         |13335  

In [23]:
final_df.select("Department", "Attrition", "profit").sort("Attrition").show(truncate = False)

+----------------------+---------+------+
|Department            |Attrition|profit|
+----------------------+---------+------+
|Research & Development|No       |9299  |
|Research & Development|No       |2623  |
|Research & Development|No       |18049 |
|Research & Development|No       |19777 |
|Research & Development|No       |-8359 |
|Research & Development|No       |20250 |
|Sales                 |No       |14348 |
|Research & Development|No       |8796  |
|Research & Development|No       |-8154 |
|Research & Development|No       |10642 |
|Sales                 |No       |3875  |
|Research & Development|No       |11340 |
|Research & Development|No       |4174  |
|Research & Development|No       |8489  |
|Research & Development|No       |12656 |
|Research & Development|No       |6097  |
|Research & Development|No       |13911 |
|Research & Development|No       |215   |
|Research & Development|No       |19278 |
|Research & Development|No       |4389  |
+----------------------+---------+

In [24]:
final_df = final_df.cache()

In [25]:
final_df.count()

1470

In [26]:
final_df.count()

1470

# 1. Using Row Number

In [27]:
final_df.select("*").sort("Department").show(truncate = False)

+--------+--------+------+---------------+-------------+-----------+---------+------+
|OverTime|OverTime|gender|Department     |MonthlyIncome|MonthlyRate|Attrition|profit|
+--------+--------+------+---------------+-------------+-----------+---------+------+
|Yes     |Yes     |Male  |Human Resources|5021         |10425      |No       |5404  |
|No      |No      |Male  |Human Resources|4936         |23965      |No       |19029 |
|Yes     |Yes     |Male  |Human Resources|2073         |23648      |Yes      |21575 |
|No      |No      |Female|Human Resources|18844        |21922      |No       |3078  |
|Yes     |Yes     |Female|Human Resources|17328        |13871      |No       |-3457 |
|No      |No      |Female|Human Resources|2942         |8916       |No       |5974  |
|Yes     |Yes     |Male  |Human Resources|6347         |13982      |No       |7635  |
|No      |No      |Male  |Human Resources|2267         |25657      |No       |23390 |
|No      |No      |Male  |Human Resources|6410        

In [28]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

In [29]:
myWindow =  Window.partitionBy("Department").orderBy(desc("MonthlyIncome"))

In [30]:
df2 =  final_df.withColumn("Number", row_number().over(myWindow)).cache()

In [31]:
df2.select("Number","Department","gender", "MonthlyIncome").show(truncate = False)

+------+----------+------+-------------+
|Number|Department|gender|MonthlyIncome|
+------+----------+------+-------------+
|1     |Sales     |Male  |19847        |
|2     |Sales     |Female|19845        |
|3     |Sales     |Male  |19833        |
|4     |Sales     |Male  |19586        |
|5     |Sales     |Female|19517        |
|6     |Sales     |Male  |19331        |
|7     |Sales     |Male  |19068        |
|8     |Sales     |Female|18947        |
|9     |Sales     |Female|18824        |
|10    |Sales     |Female|18789        |
|11    |Sales     |Male  |18303        |
|12    |Sales     |Male  |18213        |
|13    |Sales     |Female|18041        |
|14    |Sales     |Female|17875        |
|15    |Sales     |Female|17650        |
|16    |Sales     |Male  |17567        |
|17    |Sales     |Male  |17465        |
|18    |Sales     |Male  |17444        |
|19    |Sales     |Female|17048        |
|20    |Sales     |Male  |16959        |
+------+----------+------+-------------+
only showing top

# 2. Top Department By income(monthly)

In [32]:
myWindow =  Window.partitionBy("Department").orderBy(desc("MonthlyIncome"))

In [33]:
final_df.select("Department", "gender", "MonthlyIncome").show(truncate = False)

+----------------------+------+-------------+
|Department            |gender|MonthlyIncome|
+----------------------+------+-------------+
|Sales                 |Female|5993         |
|Research & Development|Male  |5130         |
|Research & Development|Male  |2090         |
|Research & Development|Female|2909         |
|Research & Development|Male  |3468         |
|Research & Development|Male  |3068         |
|Research & Development|Female|2670         |
|Research & Development|Male  |2693         |
|Research & Development|Male  |9526         |
|Research & Development|Male  |5237         |
|Research & Development|Male  |2426         |
|Research & Development|Female|4193         |
|Research & Development|Male  |2911         |
|Research & Development|Male  |2661         |
|Research & Development|Male  |2028         |
|Research & Development|Female|9980         |
|Research & Development|Male  |3298         |
|Research & Development|Male  |2935         |
|Sales                 |Female|154

In [34]:
from pyspark.sql.functions import rank

In [35]:
df3 =  final_df.select("Department", "MonthlyIncome").withColumn("rank", rank().over(myWindow)).cache()
df3.show()

+----------+-------------+----+
|Department|MonthlyIncome|rank|
+----------+-------------+----+
|     Sales|        19847|   1|
|     Sales|        19845|   2|
|     Sales|        19833|   3|
|     Sales|        19586|   4|
|     Sales|        19517|   5|
|     Sales|        19331|   6|
|     Sales|        19068|   7|
|     Sales|        18947|   8|
|     Sales|        18824|   9|
|     Sales|        18789|  10|
|     Sales|        18303|  11|
|     Sales|        18213|  12|
|     Sales|        18041|  13|
|     Sales|        17875|  14|
|     Sales|        17650|  15|
|     Sales|        17567|  16|
|     Sales|        17465|  17|
|     Sales|        17444|  18|
|     Sales|        17048|  19|
|     Sales|        16959|  20|
+----------+-------------+----+
only showing top 20 rows



In [36]:
df_salary = df3.groupBy("Department").sum("MonthlyIncome")

In [37]:
df_salary.select("Department", "sum(MonthlyIncome)").withColumnRenamed("sum(MonthlyIncome)", "salary").show(truncate = False)

+----------------------+-------+
|Department            |salary |
+----------------------+-------+
|Sales                 |3103791|
|Research & Development|6036284|
|Human Resources       |419234 |
+----------------------+-------+



In [38]:
from pyspark.sql.functions import dense_rank

In [39]:
df3 =  final_df.select("Department", "MonthlyIncome").withColumn("rank", dense_rank().over(myWindow)).cache()
df3.show()

+----------+-------------+----+
|Department|MonthlyIncome|rank|
+----------+-------------+----+
|     Sales|        19847|   1|
|     Sales|        19845|   2|
|     Sales|        19833|   3|
|     Sales|        19586|   4|
|     Sales|        19517|   5|
|     Sales|        19331|   6|
|     Sales|        19068|   7|
|     Sales|        18947|   8|
|     Sales|        18824|   9|
|     Sales|        18789|  10|
|     Sales|        18303|  11|
|     Sales|        18213|  12|
|     Sales|        18041|  13|
|     Sales|        17875|  14|
|     Sales|        17650|  15|
|     Sales|        17567|  16|
|     Sales|        17465|  17|
|     Sales|        17444|  18|
|     Sales|        17048|  19|
|     Sales|        16959|  20|
+----------+-------------+----+
only showing top 20 rows



## Lag

In [40]:
from pyspark.sql.functions import lag  

In [41]:
cumm = final_df.select("Department","gender", "MonthlyIncome").orderBy("Department")
cumm.show(truncate = False)

+---------------+------+-------------+
|Department     |gender|MonthlyIncome|
+---------------+------+-------------+
|Human Resources|Male  |5021         |
|Human Resources|Male  |4936         |
|Human Resources|Male  |2073         |
|Human Resources|Female|18844        |
|Human Resources|Female|17328        |
|Human Resources|Female|2942         |
|Human Resources|Male  |6347         |
|Human Resources|Male  |2267         |
|Human Resources|Male  |6410         |
|Human Resources|Male  |2696         |
|Human Resources|Male  |2564         |
|Human Resources|Female|9950         |
|Human Resources|Female|2741         |
|Human Resources|Male  |18200        |
|Human Resources|Female|5985         |
|Human Resources|Male  |10725        |
|Human Resources|Male  |19141        |
|Human Resources|Male  |19189        |
|Human Resources|Female|6389         |
|Human Resources|Male  |2143         |
+---------------+------+-------------+
only showing top 20 rows



In [42]:
myWindow =  Window.partitionBy("Department").orderBy(desc("MonthlyIncome"))

In [43]:
lagger = final_df.select("Department", "MonthlyIncome").withColumn("previous_value", lag("MonthlyIncome").over(myWindow))

In [44]:
lagger.show()

+----------+-------------+--------------+
|Department|MonthlyIncome|previous_value|
+----------+-------------+--------------+
|     Sales|        19847|          null|
|     Sales|        19845|         19847|
|     Sales|        19833|         19845|
|     Sales|        19586|         19833|
|     Sales|        19517|         19586|
|     Sales|        19331|         19517|
|     Sales|        19068|         19331|
|     Sales|        18947|         19068|
|     Sales|        18824|         18947|
|     Sales|        18789|         18824|
|     Sales|        18303|         18789|
|     Sales|        18213|         18303|
|     Sales|        18041|         18213|
|     Sales|        17875|         18041|
|     Sales|        17650|         17875|
|     Sales|        17567|         17650|
|     Sales|        17465|         17567|
|     Sales|        17444|         17465|
|     Sales|        17048|         17444|
|     Sales|        16959|         17048|
+----------+-------------+--------

In [45]:
comparison = lagger.withColumn("cumm_sum", lagger.MonthlyIncome + lagger.previous_value)

In [46]:
comparison.show()

+----------+-------------+--------------+--------+
|Department|MonthlyIncome|previous_value|cumm_sum|
+----------+-------------+--------------+--------+
|     Sales|        19847|          null|    null|
|     Sales|        19845|         19847|   39692|
|     Sales|        19833|         19845|   39678|
|     Sales|        19586|         19833|   39419|
|     Sales|        19517|         19586|   39103|
|     Sales|        19331|         19517|   38848|
|     Sales|        19068|         19331|   38399|
|     Sales|        18947|         19068|   38015|
|     Sales|        18824|         18947|   37771|
|     Sales|        18789|         18824|   37613|
|     Sales|        18303|         18789|   37092|
|     Sales|        18213|         18303|   36516|
|     Sales|        18041|         18213|   36254|
|     Sales|        17875|         18041|   35916|
|     Sales|        17650|         17875|   35525|
|     Sales|        17567|         17650|   35217|
|     Sales|        17465|     

## Lead

In [47]:
from pyspark.sql.functions import lead    

In [48]:
leader = final_df.select("Department","gender", "MonthlyIncome").withColumn("next", lead("MonthlyIncome").over(myWindow))
leader.show()

+----------+------+-------------+-----+
|Department|gender|MonthlyIncome| next|
+----------+------+-------------+-----+
|     Sales|  Male|        19847|19845|
|     Sales|Female|        19845|19833|
|     Sales|  Male|        19833|19586|
|     Sales|  Male|        19586|19517|
|     Sales|Female|        19517|19331|
|     Sales|  Male|        19331|19068|
|     Sales|  Male|        19068|18947|
|     Sales|Female|        18947|18824|
|     Sales|Female|        18824|18789|
|     Sales|Female|        18789|18303|
|     Sales|  Male|        18303|18213|
|     Sales|  Male|        18213|18041|
|     Sales|Female|        18041|17875|
|     Sales|Female|        17875|17650|
|     Sales|Female|        17650|17567|
|     Sales|  Male|        17567|17465|
|     Sales|  Male|        17465|17444|
|     Sales|  Male|        17444|17048|
|     Sales|Female|        17048|16959|
|     Sales|  Male|        16959|16872|
+----------+------+-------------+-----+
only showing top 20 rows



## pivot

In [49]:
cumm.show(truncate = False)

+---------------+------+-------------+
|Department     |gender|MonthlyIncome|
+---------------+------+-------------+
|Human Resources|Male  |5021         |
|Human Resources|Male  |4936         |
|Human Resources|Male  |2073         |
|Human Resources|Female|18844        |
|Human Resources|Female|17328        |
|Human Resources|Female|2942         |
|Human Resources|Male  |6347         |
|Human Resources|Male  |2267         |
|Human Resources|Male  |6410         |
|Human Resources|Male  |2696         |
|Human Resources|Male  |2564         |
|Human Resources|Female|9950         |
|Human Resources|Female|2741         |
|Human Resources|Male  |18200        |
|Human Resources|Female|5985         |
|Human Resources|Male  |10725        |
|Human Resources|Male  |19141        |
|Human Resources|Male  |19189        |
|Human Resources|Female|6389         |
|Human Resources|Male  |2143         |
+---------------+------+-------------+
only showing top 20 rows



In [50]:
from pyspark.sql.functions import sum

In [51]:
pivot_df = cumm.groupBy("Department", "gender").sum("MonthlyIncome").sort("sum(MonthlyIncome)", ascending = False)
pivot_df.show(truncate = False)

+----------------------+------+------------------+
|Department            |gender|sum(MonthlyIncome)|
+----------------------+------+------------------+
|Research & Development|Male  |3567595           |
|Research & Development|Female|2468689           |
|Sales                 |Male  |1786059           |
|Sales                 |Female|1317732           |
|Human Resources       |Male  |273954            |
|Human Resources       |Female|145280            |
+----------------------+------+------------------+



In [52]:
result = cumm.groupBy("gender").pivot("Department").agg(sum("MonthlyIncome"))
result.show()

+------+---------------+----------------------+-------+
|gender|Human Resources|Research & Development|  Sales|
+------+---------------+----------------------+-------+
|Female|         145280|               2468689|1317732|
|  Male|         273954|               3567595|1786059|
+------+---------------+----------------------+-------+



In [61]:
##########################################################
dup_schema = "Id string, Name string, Occupation string, Salary long"

In [62]:
duplicate =  spark.read.format("csv").schema(dup_schema).load("data/week08/employment.csv")
duplicate.show()

+----+-------+----------+------+
|  Id|   Name|Occupation|Salary|
+----+-------+----------+------+
|   1|Mrinmoy| Air-force|  2000|
|   2| Dhruba| Air-force|  5000|
|   3|  Subho|      Navy|  7000|
|   4|   Aabu|      Food| 20000|
|   5|    Tia|    Makeup|  6000|
|   6|   Diya|    Makeup| 10000|
|   7| Bishal|  Engineer|  1000|
|   8|   Jina|    Makeup| 20000|
|   9|  Bittu|      Navy| 20000|
|  10|   Abhi|    Fruits| 50000|
|  11|  Bishu|Vegetables|100000|
|  12|   Bidu|      Farm| 20000|
|  13|  Munna|      Farm| 20000|
|  14|  Dabbu|   Alcohol| 20001|
|  15|  Bulti|    Makeup|  9000|
|  16|  datta|  Engineer|    20|
|  17|   Abhi|   Vehicle| 30000|
|null|   null|      null|  null|
+----+-------+----------+------+



In [65]:
duplicate = duplicate.na.drop()

In [67]:
duplicate.show()

+---+-------+----------+------+
| Id|   Name|Occupation|Salary|
+---+-------+----------+------+
|  1|Mrinmoy| Air-force|  2000|
|  2| Dhruba| Air-force|  5000|
|  3|  Subho|      Navy|  7000|
|  4|   Aabu|      Food| 20000|
|  5|    Tia|    Makeup|  6000|
|  6|   Diya|    Makeup| 10000|
|  7| Bishal|  Engineer|  1000|
|  8|   Jina|    Makeup| 20000|
|  9|  Bittu|      Navy| 20000|
| 10|   Abhi|    Fruits| 50000|
| 11|  Bishu|Vegetables|100000|
| 12|   Bidu|      Farm| 20000|
| 13|  Munna|      Farm| 20000|
| 14|  Dabbu|   Alcohol| 20001|
| 15|  Bulti|    Makeup|  9000|
| 16|  datta|  Engineer|    20|
| 17|   Abhi|   Vehicle| 30000|
+---+-------+----------+------+



In [77]:
mywindow = Window.partitionBy("Occupation").orderBy("Salary")

In [78]:
duplicate.withColumn("Rank", rank().over(mywindow)).show()

+---+-------+----------+------+----+
| Id|   Name|Occupation|Salary|Rank|
+---+-------+----------+------+----+
|  1|Mrinmoy| Air-force|  2000|   1|
|  2| Dhruba| Air-force|  5000|   2|
|  4|   Aabu|      Food| 20000|   1|
| 12|   Bidu|      Farm| 20000|   1|
| 13|  Munna|      Farm| 20000|   1|
| 11|  Bishu|Vegetables|100000|   1|
| 16|  datta|  Engineer|    20|   1|
|  7| Bishal|  Engineer|  1000|   2|
| 14|  Dabbu|   Alcohol| 20001|   1|
| 10|   Abhi|    Fruits| 50000|   1|
|  3|  Subho|      Navy|  7000|   1|
|  9|  Bittu|      Navy| 20000|   2|
| 17|   Abhi|   Vehicle| 30000|   1|
|  5|    Tia|    Makeup|  6000|   1|
| 15|  Bulti|    Makeup|  9000|   2|
|  6|   Diya|    Makeup| 10000|   3|
|  8|   Jina|    Makeup| 20000|   4|
+---+-------+----------+------+----+



In [79]:
duplicate.withColumn("Dense Rank", dense_rank().over(mywindow)).show()

+---+-------+----------+------+----------+
| Id|   Name|Occupation|Salary|Dense Rank|
+---+-------+----------+------+----------+
|  1|Mrinmoy| Air-force|  2000|         1|
|  2| Dhruba| Air-force|  5000|         2|
|  4|   Aabu|      Food| 20000|         1|
| 12|   Bidu|      Farm| 20000|         1|
| 13|  Munna|      Farm| 20000|         1|
| 11|  Bishu|Vegetables|100000|         1|
| 16|  datta|  Engineer|    20|         1|
|  7| Bishal|  Engineer|  1000|         2|
| 14|  Dabbu|   Alcohol| 20001|         1|
| 10|   Abhi|    Fruits| 50000|         1|
|  3|  Subho|      Navy|  7000|         1|
|  9|  Bittu|      Navy| 20000|         2|
| 17|   Abhi|   Vehicle| 30000|         1|
|  5|    Tia|    Makeup|  6000|         1|
| 15|  Bulti|    Makeup|  9000|         2|
|  6|   Diya|    Makeup| 10000|         3|
|  8|   Jina|    Makeup| 20000|         4|
+---+-------+----------+------+----------+



In [81]:
result = duplicate.groupBy("Occupation").pivot("Name").agg(sum("Salary"))
result.show()

+----------+-----+-----+-----+------+------+-----+-----+-----+------+-----+-----+-------+-----+-----+----+-----+
|Occupation| Aabu| Abhi| Bidu|Bishal| Bishu|Bittu|Bulti|Dabbu|Dhruba| Diya| Jina|Mrinmoy|Munna|Subho| Tia|datta|
+----------+-----+-----+-----+------+------+-----+-----+-----+------+-----+-----+-------+-----+-----+----+-----+
| Air-force| null| null| null|  null|  null| null| null| null|  5000| null| null|   2000| null| null|null| null|
|      Food|20000| null| null|  null|  null| null| null| null|  null| null| null|   null| null| null|null| null|
|      Farm| null| null|20000|  null|  null| null| null| null|  null| null| null|   null|20000| null|null| null|
|Vegetables| null| null| null|  null|100000| null| null| null|  null| null| null|   null| null| null|null| null|
|  Engineer| null| null| null|  1000|  null| null| null| null|  null| null| null|   null| null| null|null|   20|
|   Alcohol| null| null| null|  null|  null| null| null|20001|  null| null| null|   null| null| 